### ЗАДАЧА: Расписание переговорок и конфликты бронирования

Есть журнал бронирований переговорных комнат в офисе.
Нужно обработать файл, найти конфликты, повторные отмены и перегруженные комнаты.

НЕОБХОДИМО РЕАЛИЗОВАТЬ:

1. Записать исходные записи в файл rooms_schedule.txt.

2. Прочитать файл rooms_schedule.txt через with open и загрузить записи в список словарей.

3. Для каждой комнаты посчитать:

   - общее количество записей

   - количество подтверждённых бронирований (`confirmed`)

   - множество сотрудников, которые бронировали комнату.

4. Найти конфликты по слотам: если для одного ключа `(date, room, slot)` больше одного `confirmed`, добавить конфликт в список.

5. Для каждого сотрудника посчитать количество подтверждённых часов (1 запись = 1 час) и множество комнат, которыми он пользовался.

6. Найти сотрудников с двойным бронированием в одно и то же время (одна дата и слот, но разные комнаты).

7. Найти комнату с наибольшим числом подтверждённых бронирований и день с максимальным количеством отмен (`cancelled`).

8. Записать подробный отчёт в файл rooms_report.txt.

In [5]:
lines = [
    "2026-03-10,Room-A,09:00,Иван,Project,confirmed",
    "2026-03-10,Room-A,09:00,Мария,HR,cancelled",
    "2026-03-10,Room-A,10:00,Олег,Sales,confirmed",
    "2026-03-10,Room-B,09:00,Иван,Project,confirmed",
    "2026-03-10,Room-B,09:00,Нина,Finance,confirmed",
    "2026-03-10,Room-C,11:00,Мария,HR,confirmed",
    "2026-03-11,Room-A,09:00,Иван,Project,confirmed",
    "2026-03-11,Room-A,09:00,Павел,Support,confirmed",
    "2026-03-11,Room-B,10:00,Мария,HR,cancelled",
    "2026-03-11,Room-B,10:00,Мария,HR,cancelled",
    "2026-03-11,Room-C,10:00,Нина,Finance,confirmed",
    "2026-03-11,Room-D,10:00,Нина,Finance,confirmed"
]

bookings = []
# список словарей: date, room, slot, employee, team, status

room_stats = {}
# room -> {"total": 0, "confirmed": 0, "employees": set()}

slot_confirmed = {}
# (date, room, slot) -> список сотрудников с confirmed (ключ-кортеж)

employee_stats = {}
# employee -> {"hours": 0, "rooms": set()}

employee_time_rooms = {}
# (date, employee, slot) -> set комнат (ключ-кортеж)

daily_cancellations = {}
# date -> количество отмен

conflicts = []
# список кортежей (date, room, slot, employees)

double_booked_employees = set()
# сотрудники с бронью в нескольких комнатах в один слот


with open("rooms_schedule.txt", "w", encoding="utf-8") as file:
    # TODO: записать lines в файл
    for line in lines:
        file.write(line + '\n')


with open("rooms_schedule.txt", "r", encoding="utf-8") as file:
    for line in file:
        line = line.strip()
        if not line:
            continue

        arr = line.split(',')
        obj = {}
        obj['date'], obj['room'], obj['slot'], obj['employee'], obj['team'], obj['status'] = arr[0], arr[1],arr[2],arr[3],arr[4],arr[5]
        bookings.append(obj)



for b in bookings:
    room_stats.setdefault(b['room'], {"total": 0, "confirmed": 0, "employees": set()})
    room_stats[b['room']]['total'] += 1
    room_stats[b['room']]['employees'].add(b['employee'])
    if b['status'] == 'confirmed':
        room_stats[b['room']]['confirmed'] += 1
        slot_confirmed.setdefault((b['date'], b['room'], b['slot']), [])
        slot_confirmed[(b['date'], b['room'],b['slot'])].append(b['employee'])

        employee_stats.setdefault(b['employee'], {"hours": 0, "rooms": set()})
        employee_stats[b['employee']]['hours'] += 1
        employee_stats[b['employee']]['rooms'].add(b['room'])

        employee_time_rooms.setdefault((b['date'],b['employee'], b['slot']), set())
        employee_time_rooms[(b['date'],b['employee'], b['slot'])].add(b['room'])
    elif b['status'] =='cancelled':
        daily_cancellations.setdefault(b['date'], 0)
        daily_cancellations[b['date']] += 1


for key, employees in slot_confirmed.items():
    if len(employees) > 1:
        conflicts.append(tuple(key) + tuple(employees))


for (date, employee, slot), rooms in employee_time_rooms.items():

    if len(rooms) > 1:
        double_booked_employees.add(employee)

top_room = None
top_room_confirmed = -1


for k in room_stats.keys():
    if room_stats[k]['confirmed'] > top_room_confirmed:
        top_room_confirmed = room_stats[k]['confirmed']
        top_room = k

cancel_day = None
cancel_day_count = -1

for k in daily_cancellations.keys():
    if daily_cancellations[k] > cancel_day_count:
        cancel_day_count = daily_cancellations[k]
        cancel_day = k

print(room_stats)
with open("rooms_report.txt", "w", encoding="utf-8") as file:

    file.write(str(room_stats) + '\n')
    file.write(str(daily_cancellations) + '\n')
    file.write(str(employee_stats) + '\n')
    file.write('сотрудники с двойным бронированием: ' + str(double_booked_employees)+'\n')
    m = 0
    day = None
    for k in daily_cancellations:
        if m < daily_cancellations[k]:
            m = daily_cancellations[k]
            day = k
    file.write(f'топ-комната: {top_room}, день с максимумом отмен: {day}')

{'Room-A': {'total': 5, 'confirmed': 4, 'employees': {'Павел', 'Олег', 'Мария', 'Иван'}}, 'Room-B': {'total': 4, 'confirmed': 2, 'employees': {'Мария', 'Нина', 'Иван'}}, 'Room-C': {'total': 2, 'confirmed': 2, 'employees': {'Мария', 'Нина'}}, 'Room-D': {'total': 1, 'confirmed': 1, 'employees': {'Нина'}}}
